In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/Neighborhood_Zhvi_AllHomes.csv')
df.head(6)

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12
0,274772,Northeast Dallas,Dallas,TX,Dallas-Fort Worth-Arlington,Dallas County,1,135202.0,135581.0,135338.0,...,325161,325900,325727,325381,325511,325787,327295,325192,323607,320725
1,112345,Maryvale,Phoenix,AZ,Phoenix-Mesa-Scottsdale,Maricopa County,2,NaN,NaN,NaN,...,179434,180603,181465,182230,183329,184639,185839,186460,186891,187899
2,192689,Paradise,Las Vegas,NV,Las Vegas-Henderson-Paradise,Clark County,3,139698.0,139598.0,139550.0,...,267175,267837,268027,268152,267129,265625,264568,264910,266447,267638
3,270958,Upper West Side,New York,NY,New York-Newark-Jersey City,New York County,4,254172.0,255211.0,255804.0,...,1302571,1286609,1285449,1280555,1269061,1247156,1226245,1225663,1243710,1263182
4,118208,South Los Angeles,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles County,5,134761.0,135045.0,135163.0,...,498385,500046,502408,505005,503896,503075,503662,508421,512767,518271
5,270957,Upper East Side,New York,NY,New York-Newark-Jersey City,New York County,6,NaN,NaN,NaN,...,998091,990259,985902,977501,976390,975670,976743,972887,978099,985448


In [3]:
# Get names of date columns for selection
date_names = df.columns[7:]
date_names

Index(['1996-04', '1996-05', '1996-06', '1996-07', '1996-08', '1996-09',
       '1996-10', '1996-11', '1996-12', '1997-01',
       ...
       '2019-03', '2019-04', '2019-05', '2019-06', '2019-07', '2019-08',
       '2019-09', '2019-10', '2019-11', '2019-12'],
      dtype='object', length=285)

In [4]:
# Subset data frame and transpose
df_transpose = df[date_names]
df_transpose = df_transpose.transpose()
df_transpose.head()

,0,1,2,3,4,5,6,7,8,9,...,16154,16155,16156,16157,16158,16159,16160,16161,16162,16163
1996-04,135202.0,NaN,139698.0,254172.0,134761.0,NaN,NaN,148924.0,NaN,152565.0,...,98064.0,73017.0,102232.0,352258.0,75026.0,NaN,74229.0,104551.0,62338.0,101059.0
1996-05,135581.0,NaN,139598.0,255211.0,135045.0,NaN,NaN,148746.0,NaN,151706.0,...,96950.0,74118.0,101251.0,351343.0,75214.0,NaN,75750.0,105812.0,62841.0,101063.0
1996-06,135338.0,NaN,139550.0,255804.0,135163.0,NaN,NaN,148543.0,NaN,151262.0,...,97549.0,75170.0,101840.0,352595.0,75149.0,NaN,76987.0,105342.0,63409.0,102104.0
1996-07,134586.0,NaN,139602.0,254853.0,135109.0,NaN,NaN,148543.0,NaN,150954.0,...,96755.0,75303.0,102159.0,353868.0,76123.0,NaN,77692.0,104822.0,64077.0,102511.0
1996-08,134420.0,NaN,140131.0,254897.0,135203.0,NaN,NaN,149059.0,NaN,151225.0,...,97420.0,75364.0,102951.0,357397.0,76125.0,NaN,78291.0,102740.0,64529.0,102563.0


In [5]:
# convert index to datetime and run median average per year
df_transpose.index = pd.to_datetime(df_transpose.index, format='%Y-%m')

With time series data, using pad/ffill is extremely common so that the “last known value” is available at every time point.

### The simplest method would be to look at median house price per neighborhood for one timeslice
lets calculate a column for that first

In [6]:
# get average median housing prices for time slice 2015-2019
def median_2015_2019(row):
    year = df_transpose[row].resample('Y', axis='index').mean().ffill()
    year = year[19:].mean()
    return year

In [7]:
df['median_2015_2019'] = df.index.to_series().apply(median_2015_2019).round(2)
df.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,median_2015_2019
0,274772,Northeast Dallas,Dallas,TX,Dallas-Fort Worth-Arlington,Dallas County,1,135202.0,135581.0,135338.0,...,325900,325727,325381,325511,325787,327295,325192,323607,320725,280025.77
1,112345,Maryvale,Phoenix,AZ,Phoenix-Mesa-Scottsdale,Maricopa County,2,NaN,NaN,NaN,...,180603,181465,182230,183329,184639,185839,186460,186891,187899,153498.08
2,192689,Paradise,Las Vegas,NV,Las Vegas-Henderson-Paradise,Clark County,3,139698.0,139598.0,139550.0,...,267837,268027,268152,267129,265625,264568,264910,266447,267638,222027.07
3,270958,Upper West Side,New York,NY,New York-Newark-Jersey City,New York County,4,254172.0,255211.0,255804.0,...,1286609,1285449,1280555,1269061,1247156,1226245,1225663,1243710,1263182,1267720.98
4,118208,South Los Angeles,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles County,5,134761.0,135045.0,135163.0,...,500046,502408,505005,503896,503075,503662,508421,512767,518271,428261.07


### Another metric would be rate of change/growth rate as a proxy for gentrification 
but I'm not sure if this works with the yelp data - would I have to calculate rate of change for those metrics too? Or at least have some time-series information. I'm going to calculate a column for that in case I want to use it. For this one let's do 2010-2019

In [8]:
# Compound growth rate formula - not sure if I need to use this for home price
def cagr(start_value, end_value, num_periods):
    return (end_value / start_value) ** (1 / (num_periods)) - 1

def ca_growth_rate(row):
    #row = df.index[df['RegionID'] == RegionID]
    year = df_transpose[row].resample('Y', axis='index').mean().ffill()
    year_2010_2019 = year[14:]
    start = float(year_2010_2019.iloc[0])
    end = float(year_2010_2019.iloc[-1])
    periods = len(year_2010_2019)
    gr = cagr(start, end, periods)
    return gr

In [9]:
# Percent change between 2019 and 2010
def growth_rate(row):
    year = df_transpose[row].resample('Y', axis='index').mean().ffill()
    year_2010_2019 = pd.Series([year['2010-12-31'], year['2019-12-31']])
    change = year_2010_2019.pct_change()[1]
    return change

In [10]:
df['change_2010_2019'] = df.index.to_series().apply(growth_rate)
df['avg_growth_rate_2010_2019'] = df.index.to_series().apply(ca_growth_rate)
df.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,median_2015_2019,change_2010_2019,avg_growth_rate_2010_2019
0,274772,Northeast Dallas,Dallas,TX,Dallas-Fort Worth-Arlington,Dallas County,1,135202.0,135581.0,135338.0,...,325381,325511,325787,327295,325192,323607,320725,280025.77,0.749869,0.057549
1,112345,Maryvale,Phoenix,AZ,Phoenix-Mesa-Scottsdale,Maricopa County,2,NaN,NaN,NaN,...,182230,183329,184639,185839,186460,186891,187899,153498.08,1.515827,0.096650
2,192689,Paradise,Las Vegas,NV,Las Vegas-Henderson-Paradise,Clark County,3,139698.0,139598.0,139550.0,...,268152,267129,265625,264568,264910,266447,267638,222027.07,1.009707,0.072293
3,270958,Upper West Side,New York,NY,New York-Newark-Jersey City,New York County,4,254172.0,255211.0,255804.0,...,1280555,1269061,1247156,1226245,1225663,1243710,1263182,1267720.98,0.476743,0.039754
4,118208,South Los Angeles,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles County,5,134761.0,135045.0,135163.0,...,505005,503896,503075,503662,508421,512767,518271,428261.07,1.033195,0.073539


In [12]:
# reduce columns and save as csv
changes_df = df[['RegionID', 'RegionName', 'City', 'State', 'Metro']].join(
    df[['median_2015_2019', 'change_2010_2019', 'avg_growth_rate_2010_2019']])
changes_df.to_csv('data/house_price_changes.csv')

### Zillow has data on affordabiliy but it's by City not by neighborhood

In [13]:
affordability = pd.read_csv('data/Affordability_Wide_2019Q3_Public.csv')
affordability[affordability['RegionName'] == 'San Francisco, CA']

,RegionID,RegionName,SizeRank,Index,HistoricAverage_1985thru1999,1979-03,1979-06,1979-09,1979-12,1980-03,...,2017-06,2017-09,2017-12,2018-03,2018-06,2018-09,2018-12,2019-03,2019-06,2019-09
11,395057,"San Francisco, CA",11,Price To Income,5.002622,4.253145,4.382501,4.572843,4.703875,4.834304,...,8.558171,8.555372,8.613288,8.951910,9.064145,9.200361,9.199402,9.133212,9.027692,8.966299
380,395057,"San Francisco, CA",11,Mortgage Affordability,0.384347,0.371355,0.402889,0.428045,0.496002,0.597224,...,0.387515,0.383166,0.392383,0.432379,0.444523,0.454370,0.454852,0.431841,0.403826,0.391827
749,395057,"San Francisco, CA",11,Rent Affordability,0.312387,0.289501,0.287501,0.284881,0.282976,0.279761,...,0.368852,0.366968,0.356964,0.354777,0.356849,0.358575,0.357654,0.358805,0.362604,0.364676
